Compare House TCJA total change in liability when using default weights vs. those provided by Ernie Tedeschi. 

Uses behavior file provided by Matt Jensen.

In [1]:
from taxcalc import *
from taxcalc.utils import *

In [2]:
# Load Ernie Tedeschi's weights file.
tedeschi_weights = pd.read_csv('~/cps_weights_2026.csv')

In [98]:
tedeschi_weights.size

912930

In [67]:
def print_total_weight(v, msg):
    print 'Total record weight [' + msg + ']: {:0.2f}'.format(v.sum())

What the total record weight should be.

In [70]:
print_total_weight(Records.cps_constructor().s006, 'Records raw')
print_total_weight(tedeschi_weights.s006, 'Tedeschi raw')

Total record weight [Records raw]: 163195770.40
Total record weight [Tedeschi raw]: 191164588.00


In [33]:
def total_combined_liability(calc):
    # combined is combined tax liability while s006 is weight.
    return (calc.records.combined * calc.records.s006).sum()

In [34]:
def print_combined_liability(baseline_calc, reform_calc):
    # Calculate and print liabilities.
    baseline = total_combined_liability(baseline_calc)
    reformed = total_combined_liability(reform_calc)
    diff = reformed - baseline
    print ('Combined Liability - Baseline: {:0.2f}'.format(baseline))
    print ('Combined Liability - Reform: {:>18.2f}'.format(reformed))
    print ('-' * 47)
    print ('Difference: {:35.2f}'.format(diff))

In [92]:
def test(replace_records_tedeschi_weights,
         replace_calc_tedeschi_weights,
         replace_calc_tedeschi_weights_after_advance_to_year):
    # Initiate baseline calculator and replace weights as needed.
    recs = Records.cps_constructor()
    if (replace_records_tedeschi_weights):
        recs.s006 = tedeschi_weights.s006
    print_total_weight(recs.s006, 'recs after setting record weight')
    calc = Calculator(records=recs, policy=Policy())
    print_total_weight(calc.records.s006, 'calc after creation')
    if (replace_calc_tedeschi_weights):
        calc.records.s006 = tedeschi_weights.s006
    print_total_weight(calc.records.s006, 'calc after setting calc weight')
    # Advance and calculate.
    calc.advance_to_year(2018)
    if (replace_calc_tedeschi_weights_after_advance_to_year):
        calc.records.s006 = tedeschi_weights.s006
    print_total_weight(calc.records.s006, 'calc after advance_to_year')
    calc.calc_all()
    print_total_weight(calc.records.s006, 'calc after calc_all')
    # Initiate calculator to apply reforms to
    reform = Calculator.read_json_param_objects('TCJA_House_Amended.json',
                                                'JCT_Behavior.json')
    pol = Policy()
    pol.implement_reform(reform['policy'])
    con = Consumption()
    con.update_consumption(reform['consumption'])
    beh = Behavior()
    beh.update_behavior(reform['behavior'])
    calc_x = Calculator(records=recs, policy=pol,
                        consumption=con, behavior=beh)
    print_total_weight(calc_x.records.s006, 'calc_x after creation')
    if (replace_calc_tedeschi_weights):
        calc_x.s006 = tedeschi_weights.s006
    calc_x.advance_to_year(2018)
    if (replace_calc_tedeschi_weights_after_advance_to_year):
        calc_x.records.s006 = tedeschi_weights.s006
    print_total_weight(calc_x.records.s006, 'calc_x after advance_to_year')
    calc_response = Behavior.response(calc, calc_x)
    print_total_weight(calc_response.records.s006, 
                       'calc_response after creation')
    print_combined_liability(calc, calc_response)

In [93]:
test(replace_records_tedeschi_weights = True,
     replace_calc_tedeschi_weights = True,
     replace_calc_tedeschi_weights_after_advance_to_year = False)

Total record weight [recs after setting record weight]: 191164588.00
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
Total record weight [calc after creation]: 191164588.00
Total record weight [calc after setting calc weight]: 191164588.00
Total record weight [calc after advance_to_year]: 169894839.66
Total record weight [calc after calc_all]: 169894839.66
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
Total record weight [calc_x after creation]: 191164588.00
Total record weight [calc_x after advance_to_year]: 169894839.66
Total record weight [calc_response after creation]: 169894839.66
Combined Liability - Baseline: 2521070511449.64
Combined Liability - Reform:   2458101656483.74
-----------------------------------------------
Difference:                     -62968854965.90


In [94]:
test(replace_records_tedeschi_weights = True,
     replace_calc_tedeschi_weights = True,
     replace_calc_tedeschi_weights_after_advance_to_year = True)

Total record weight [recs after setting record weight]: 191164588.00
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
Total record weight [calc after creation]: 191164588.00
Total record weight [calc after setting calc weight]: 191164588.00
Total record weight [calc after advance_to_year]: 191164588.00
Total record weight [calc after calc_all]: 191164588.00
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
Total record weight [calc_x after creation]: 191164588.00
Total record weight [calc_x after advance_to_year]: 191164588.00
Total record weight [calc_response after creation]: 191164588.00
Combined Liability - Baseline: 7458477789103.90
Combined Liability - Reform:   7412113754574.96
-----------------------------------------------
Difference:                     -46364034528.94


In [95]:
test(replace_records_tedeschi_weights = False,
     replace_calc_tedeschi_weights = False,
     replace_calc_tedeschi_weights_after_advance_to_year = False)

Total record weight [recs after setting record weight]: 163195770.40
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
Total record weight [calc after creation]: 163195770.40
Total record weight [calc after setting calc weight]: 163195770.40
Total record weight [calc after advance_to_year]: 169894839.66
Total record weight [calc after calc_all]: 169894839.66
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
Total record weight [calc_x after creation]: 163195770.40
Total record weight [calc_x after advance_to_year]: 169894839.66
Total record weight [calc_response after creation]: 169894839.66
Combined Liability - Baseline: 2521070511449.64
Combined Liability - Reform:   2458101656483.74
-----------------------------------------------
Difference:                     -62968854965.90
